In [1]:

import torch
import os
import matplotlib.pyplot as plt

from torchvision.io import read_image, write_png
from torch.utils.cpp_extension import load_inline
from pathlib import Path

In [2]:
torch.__version__

'2.0.1+cu117'

In [8]:
DEVICE = "cuda:0"
ROOT_DIR = os.path.abspath("..")
IMAGES_DIR = Path(ROOT_DIR) / "images"
BUILD_DIR = Path(ROOT_DIR) / "build"
KERNELS_DIR = Path(ROOT_DIR) / "kernels"
IMAGES_DIR


PosixPath('/media/nvme/vs_code/cuda_torch_learning/pmpp/images')

In [ ]:
test_image = read_image(str(IMAGES_DIR / "test_image.png")).to(DEVICE).float()

### Image Blur

In [7]:
cuda_source = (KERNELS_DIR / "imageBlur.cu").read_text()
cpp_source = "torch::Tensor blur_filter(torch::Tensor image, int radius);"
image_blur_module = load_inline(
    name="image_blur",
    cpp_sources=[cpp_source],
    cuda_sources=[cuda_source],
    functions=["blur_filter"],
    # build_directory=BUILD_DIR,
    verbose=True,
)

Using /home/aleksei/.cache/torch_extensions/py310_cu117 as PyTorch extensions root...
The input conditions for extension module image_blur have changed. Bumping to version 2 and re-building as image_blur_v2...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/aleksei/.cache/torch_extensions/py310_cu117/image_blur/build.ninja...
Building extension module image_blur_v2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=image_blur_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/aleksei/miniconda3/envs/classification-dev2/lib/python3.10/site-packages/torch/include -isystem /home/aleksei/miniconda3/envs/classification-dev2/lib/python3.10/site-packages/torch/include/torch/csrc/api/include -isystem /home/aleksei/miniconda3/envs/classification-dev2/lib/python3.10/site-packages/torch/include/TH -isystem /home/aleksei/miniconda3/envs/classification-dev2/lib/python3.10/site-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /home/aleksei/miniconda3/envs/classification-dev2/include/python3.10 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/aleksei/.cache/torch_extensions/py310_cu117/image_blur/main.cpp -o main.o 
[2/3] /usr/local/cuda/bin/nvcc  -DTORCH_EXTENSION_NAME=image_blur_v2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_T

Loading extension module image_blur_v2...


In [11]:
test_random_tensor = torch.ones(3, 64, 64, device=DEVICE, dtype=torch.uint8)
test_blur = image_blur_module.blur_filter(test_random_tensor, 5)

In [12]:
test_blur

tensor([[[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]], device='cuda:0', dtype=torch.uint8)